# Neuro UNet/ MeshnetTutorial

Authors: [Kevin Wang] (), [Alex Fedorov] (), [Sergey Kolesnikov](https://github.com/Scitator)

[![Catalyst logo](https://raw.githubusercontent.com/catalyst-team/catalyst-pics/master/pics/catalyst_logo.png)](https://github.com/catalyst-team/catalyst)

### Colab setup

First of all, do not forget to change the runtime type to GPU. <br/>
To do so click `Runtime` -> `Change runtime type` -> Select `\"Python 3\"` and `\"GPU\"` -> click `Save`. <br/>
After that you can click `Runtime` -> `Run all` and watch the tutorial.

## Requirements

Download and install the latest versions of catalyst and other libraries required for this tutorial.

In [ ]:
from typing import Callable, List, Tuple

import os
import torch
import catalyst
from catalyst import utils

print(f"torch: {torch.__version__}, catalyst: {catalyst.__version__}")

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # "" - CPU, "0" - 1 GPU, "0,1" - MultiGPU

SEED = 42
utils.set_global_seed(SEED)
utils.prepare_cudnn(deterministic=True)

# Dataset

We'll be using the Mindboggle 101 dataset for a multiclass 3d segmentation task.
The dataset can be downloaded off osf with the following command from osfclient after you register with osf.

`osf -p 9ahyp clone .`

Otherwise you can download it using a Catalyst utility `download-gdrive` which downloads a version from the Catalyst Google Drive

`usage: download-gdrive {FILE_ID} {FILENAME}`

In [ ]:
mkdir Mindboggle_data 

In [ ]:
%%bash 

# todo: make download-gdrive here
osf -p 9ahyp clone Mindboggle_data/

Copy and extract volumes to the following location.

In [ ]:
cp -r Mindboggle_data/osfstorage/Mindboggle101_volumes/ ../data/Mindboggle_data/
find data/Mindboggle_101 -name '*.tar.gz'| xargs -i tar zxvf {} -C data/Mindboggle_101
find data/Mindboggle_101 -name '*.tar.gz'| xargs -i rm {}

Run the prepare data script that limits the labels to 30 labels.

`usage: python ../neuro/scripts/prepare_data.py ../data/Mindboggle_101 {N_labels)`

In [ ]:
%%bash 

python ../neuro/scripts/prepare_data.py ../data/Mindboggle_101/

Import Catalyst and Torch utils for training

In [ ]:
import torch
import collections

from multiprocessing import Manager
from catalyst.contrib.utils.pandas import read_csv_data
from torch.utils.data import RandomSampler
from torch.utils.data import DataLoader
from torchvision import transforms
from catalyst.data import Augmentor, ReaderCompose

Here we import a BrainDataSet, which reads T1 scans and labels and samples either random patches of 38x38x38 samples from them or nonoverlapping patches of 38x38x38 for validation.  More detail can be found in brain_dataset.py and generator_coords.py  

In [ ]:
from brain_dataset import BrainDataset
from reader import NiftiReader_Image, NiftiReader_Mask

The Transforms for the BrainDataset here are simple. 

Convert the T1 scans from numpy arrays to PyTorch floats and convert the corresponding labels to whatever default Torch array exists.

In [ ]:
def get_transforms(stage: str = None, mode: str = None):
    if mode == "train":                                                                                                                                                                                 
        Augmentor1 = Augmentor(                                                                                                                                                                         
            dict_key="images",                                                                                                                                                                          
            augment_fn=lambda x: torch.from_numpy(x).float(),                                                                                                                                           
        )                                                                                                                                                                                               
        Augmentor2 = Augmentor(                                                                                                                                                                         
            dict_key="targets", augment_fn=lambda x: torch.from_numpy(x)                                                                                                                                
        )                                                                                                                                                                                               
        return transforms.Compose([Augmentor1, Augmentor2])                                                                                                                                             
    elif mode == "valid":                                                                                                                                                                               
        Augmentor1 = Augmentor(                                                                                                                                                                         
            dict_key="images",                                                                                                                                                                          
            augment_fn=lambda x: torch.from_numpy(x).float(),                                                                                                                                           
        )                                                                                                                                                                                               
        Augmentor2 = Augmentor(                                                                                                                                                                         
            dict_key="targets", augment_fn=lambda x: torch.from_numpy(x)                                                                                                                                
        )                                                                                                                                                                                               
        return transforms.Compose([Augmentor1, Augmentor2])

In [ ]:
open_fn = ReaderCompose(                                                                                                                                                                            
    readers=[                                                                                                                                                                                       
        NiftiReader_Image(input_key="images", output_key="images"),                                                                                                                                 
        NiftiReader_Mask(input_key="nii_labels", output_key="targets"),
    ]
)

In [ ]:
def get_loaders(
    volume_shape: List[int],
    subvolume_shape: List[int],
    in_csv_train: str = None,
    in_csv_valid: str = None,
    in_csv_infer: str = None,
    batch_size: int = 16,
    num_workers: int = 10,
) -> dict:

    df, df_train, df_valid, df_infer = read_csv_data(
        in_csv_train=in_csv_train,
        in_csv_valid=in_csv_valid,
        in_csv_infer=in_csv_infer,
    )

    train_dataset = BrainDataset(
        shared_dict={},
        list_data=df_train,
        list_shape=volume_shape,
        list_sub_shape=subvolume_shape,
        open_fn=open_fn,
        dict_transform=get_transforms(None, mode="train"),
        n_samples=100,
        mode="train",
        input_key="images",
        output_key="targets",
    )
    valid_dataset = BrainDataset(
        shared_dict={},
        list_data=df_valid,
        list_shape=volume_shape,
        list_sub_shape=subvolume_shape,
        open_fn=open_fn,
        dict_transform=get_transforms(None, mode="valid"),
        n_samples=100,
        mode="valid",
        input_key="images",
        output_key="targets",
    )
    # test_dataset = BrainDataset(
    #     shared_dict={},
    #     list_data=df_infer,
    #     list_shape=volume_shape,
    #     list_sub_shape=subvolume_shape,
    #     open_fn=open_fn,
    #     dict_transform=get_transforms(None, mode="valid"),
    #     n_samples=100,
    #     mode="valid",
    #     input_key="images",
    #     output_key="targets",
    # )

    train_random_sampler = RandomSampler(
        data_source=train_dataset, replacement=True, num_samples=80 * 128
    )

    valid_random_sampler = RandomSampler(
        data_source=valid_dataset, replacement=True, num_samples=20 * 216
    )

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        sampler=train_random_sampler,
        num_workers=num_workers,
        pin_memory=True,
    )
    valid_loader = DataLoader(
        dataset=valid_dataset,
        batch_size=batch_size,
        sampler=valid_random_sampler,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=True,
    )
    loaders = collections.OrderedDict()
    loaders["train"] = train_loader
    loaders["valid"] = valid_loader

    return loaders


In [ ]:
loaders = get_loaders(
    0,
    [256, 256, 256],
    [38, 38, 38],
    "../data/dataset_train.csv",
    "../data/dataset_valid.csv",
    "../data/dataset_infer.csv",
)
train_dataloader = loaders["train"]
next(iter(train_dataloader))

# Model

We'll be using the UNet defined in the model.py file for training

In [ ]:
from model import UNet

unet = UNet(n_channels=1, n_classes=30)

# Model Training

We'll train the model 30 epochs

An Adam Optimizer with a cosine annealing schedule starting at a learning rate of .01 is used for this experiment.

CrossEntropyLoss is the criterion/ loss function be minimized 

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import CosineAnnealingLR
from catalyst.dl import SupervisedRunner
from catalyst.callbacks.logging import TensorboardLogger
from catalyst.callbacks import SchedulerCallback, CheckpointCallback
from custom_metrics import CustomDiceCallback

num_epochs = 30
logdir = "logs/unet"

optimizer = torch.optim.Adam(unet.parameters(), lr=0.01, weight_decay=0.0001)
scheduler = CosineAnnealingLR(optimizer, T_max=30)

runner = SupervisedRunner(
    input_key="images", input_target_key="labels", output_key="logits"
)

callbacks = [
    TensorboardLogger(),
    SchedulerCallback(reduced_metric="loss"),
    CustomDiceCallback(),
    CheckpointCallback(),
]

runner.train(
    model=unet,
    criterion=CrossEntropyLoss(),
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    callbacks=callbacks,
    logdir=logdir,
    num_epochs=num_epochs,
    verbose=True,
)
